# Now detecting those Parcels/ Khasras of Pabbi which has now been converted to buildings.
Steps:
- Load Pabbi vector dataset and filterout the parcels which were originally agricultrue now has built up structure
- Load building_dataset.geoparquet to detect buildings in these parcels
- Visualize them.

In [37]:
import geoai
import leafmap
import geemap
import geopandas as gpd
import ee

In [11]:
ee.Authenticate()


Successfully saved authorization token.


In [38]:
geemap.ee.Initialize()

In [39]:
#parquet_url = 'https://github.com/code4geoai/gee/releases/download/0.2/pabbi.parquet'
parquet_file = 'pabbi.parquet'
# The parquet file must be local or can be read from url with request. lets use the local file.
gdf = gpd.read_parquet(parquet_file)


In [131]:
gdf.value_counts()

Mouza_Name  Landuse_Ma   Area_Acre  FFID  Parcel_ID  geometry                                         
Khushmaqam  Stream       1.232730   116   649.0      MULTIPOLYGON (((754225.322 3769596.568, 754215...    1
Amankot     Agriculture  0.004193   6623  582.0      MULTIPOLYGON (((754519.923 3769858.42, 754516....    1
                         0.011662   6518  761.0      MULTIPOLYGON (((754832.496 3770130.628, 754826...    1
                         0.017473   6526  646.0      MULTIPOLYGON (((754667.98 3770199.197, 754663....    1
                         0.020042   6222  834.0      MULTIPOLYGON (((755434.675 3770424.439, 755411...    1
                                                                                                         ..
                         0.034051   6643  285.0      MULTIPOLYGON (((754915.798 3769345.421, 754903...    1
                         0.034832   5813  73.0       MULTIPOLYGON (((754314.423 3769802.321, 754311...    1
                         0.035476

In [113]:
len(gdf)

6740

In [40]:
#Converted the pabbi dataframe to google earth engine feature collection.
pabbi= geemap.gdf_to_ee(gdf)

In [82]:
#Filtering the features having Landuse_Ma as Agriculture
agri_parcel=gdf[gdf['Landuse_Ma'] == 'Agriculture']
agri_parcel.value_counts()

Mouza_Name  Landuse_Ma   Area_Acre  FFID  Parcel_ID  geometry                                         
Khushmaqam  Agriculture  7.358120   448   197.0      MULTIPOLYGON (((753417.793 3770106.787, 753421...    1
Amankot     Agriculture  0.004193   6623  582.0      MULTIPOLYGON (((754519.923 3769858.42, 754516....    1
                         0.011662   6518  761.0      MULTIPOLYGON (((754832.496 3770130.628, 754826...    1
                         0.017473   6526  646.0      MULTIPOLYGON (((754667.98 3770199.197, 754663....    1
                         0.020042   6222  834.0      MULTIPOLYGON (((755434.675 3770424.439, 755411...    1
                                                                                                         ..
                         0.041556   6626  757.0      MULTIPOLYGON (((754866.603 3770058.608, 754865...    1
                         0.043187   4530  1822.0     MULTIPOLYGON (((757240.969 3768293.951, 757222...    1
                         0.043764

In [ ]:
#Converting Agri dataframe to google earth engine
agri_feature = geemap.gdf_to_ee(agri_parcel)



In [56]:
#Step 1 Processing satellite Imagery

# Adding Satellite Imagery and calculating NDVI using Sentinel-2
collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
# Filter the collection for a specific region and time period
start_date = '2021-01-01'
end_date = '2024-12-31'
# Filter the collection by date and region
filtered_collection = collection.filterDate(start_date, end_date).filterBounds(pabbi.geometry()).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))

# Select the first image from the filtered collection
median_image = filtered_collection.median().clip(pabbi.geometry())

# Calculate NDVI
ndvi = median_image.normalizedDifference(['B8', 'B4']).rename('NDVI')


# Select the bands of interest (e.g.,NVDI)
ndvi_vis = {
    'min': 0.25,
    'max': 0.8,
    'palette': ['white', 'yellow', 'green', 'red']  # Red for highest vegetation
}


In [ ]:
filtered_collection.size() # Get the size of the image in bytes

# for one year i.e. 2024 the number of images is 135 and for three years i.e. from 2021 to 2024 the number of images is 604

In [ ]:
# Filtering out the Agricultue parcles which are now built up 
# Step 1: Calculate NDVI for entire area alrady done above

# Step 2: Calculate mean NDVI for each agri polygon
agri_with_ndvi = ndvi.reduceRegions(
    collection=agri_feature,
    reducer=ee.Reducer.mean(),
    scale=10,
)


# Step 3: Create a feature collection by Filterout polygons with NDVI less than 0.3
# 
agri_to_builtup = agri_with_ndvi.filter(ee.Filter.lt('mean', 0.3))


1227

In [ ]:
#Converting the filtered agri_to_builtup feature collection  to geodataframe
builtup_parcels = geemap.ee_to_gdf(agri_to_builtup)


In [ ]:
# No of parcles converted from agriculture to builtup
agri_to_builtup.size()
# Performance has been improved and number of builtup parcels has been reduced from 1279 to 1227.
# 1227 are the potential parcels which have been converted from agriculture to builtup.

# Part 2
## Working with building dataset geoparquet file

In [101]:
# Now load the building parquet file and filter the buildings which are in agriculture area.
parquet_file = 'building_dataset.geoparquet'
# reading the parquet file
building = gpd.read_parquet(parquet_file)


In [133]:
building.count()

id                        36806
geometry                  36806
theme                     36806
type                      36806
version                   36806
sources                   36806
level                         0
subtype                       0
class                         0
height                        0
names                         0
has_parts                 36806
is_underground            36806
num_floors                    0
num_floors_underground        0
min_height                    0
min_floor                     0
facade_color                  0
facade_material               0
roof_material                 0
roof_shape                    0
roof_direction                0
roof_orientation              0
roof_color                    0
roof_height                   0
dtype: int64

In [103]:
from shapely.ops import unary_union

# Step 1: Create a single unified geometry from all building polygons
building_union = unary_union(building.geometry)

# Step 2: Filter builtup_parcels that intersect the building_union
intersected_parcels = builtup_parcels[builtup_parcels.geometry.intersects(building_union)]
intersected_parcels = intersected_parcels.reset_index(drop=True)
intersected_parcels.count()



geometry      1144
Area_Acre     1144
FFID          1144
Landuse_Ma    1144
Mouza_Name    1144
Parcel_ID     1144
mean          1144
dtype: int64

## BuiltUp Parcels:
# Out of the potential 1227 parcels, 1144 parcels are confirmed with buildings.

### Part 3: Visualizing Building footprint

In [139]:
# Ensure both GeoDataFrames are in the same CRS
building = building.to_crs(gdf.crs)

# Spatial join — keep only buildings that intersect with gdf polygons
buildings_in_gdf = gpd.sjoin(building, gdf, how="inner", predicate="intersects")

buildings_in_gdf.count()

id                        20372
geometry                  20372
theme                     20372
type                      20372
version                   20372
sources                   20372
level                         0
subtype                       0
class                         0
height                        0
names                         0
has_parts                 20372
is_underground            20372
num_floors                    0
num_floors_underground        0
min_height                    0
min_floor                     0
facade_color                  0
facade_material               0
roof_material                 0
roof_shape                    0
roof_direction                0
roof_orientation              0
roof_color                    0
roof_height                   0
index_right               20372
Mouza_Name                20372
Landuse_Ma                19791
Area_Acre                 20372
FFID                      20372
Parcel_ID                 20372
dtype: i

In [142]:
agristyle ={"color": "lightgreen", "fillColor": " ", "fillOpacity": 0.1, "weight": 2}

pbuiltupstyle = {"color": "yellow", "fillColor": "yellow", "fillOpacity": 0.1, "weight": 2}

buildingstyle ={"color": "red", "fillColor": "red", "fillOpacity": 0.1, "weight": 2}

bfootprintstyle ={"color": "blue", "fillColor": "blue", "fillOpacity": 0.4, "weight": 1}

In [ ]:
# Plotting agri_features and agri_to_builtup
m = leafmap.Map(zoom_control=False,
    draw_control=False,
    scale_control=False,
    fullscreen_control=False,)
m.add_layer_control()
m.add_basemap('Google Satellite')

m.add_gdf(agri_parcel,style=agristyle,layer_name= 'Massavi Agri Parcels')
# visualizing the building footprint
m.add_gdf(buildings_in_gdf, style=bfootprintstyle, layer_name='Building Footprint')
m.add_gdf(intersected_parcels, style=buildingstyle, layer_name='Builtup Parcels',zoom_to_layer=True)

#m.add_gdf(builtup_parcels, style=pbuiltupstyle, layer_name='Potential Builtup Parcels',zoom_to_layer=True) 




m


Map(center=[20, 0], controls=(AttributionControl(options=['position', 'prefix'], position='bottomright'), Widg…